## Connecting Cleaned Data with the Closest Station Name 

*Note: this file original merged on Lat Long, After some code modification to the file that creates the noaa station matches, It can now merge on row number

1. Load in the newest data cbc data: "bird_count_cleaned_may_29_2019.csv"
2. Drop all the circles that are not in the United States
3. Clean Up lat long data
DEPRECATED: 4. Merge on Lat Long Location  
4. Merge on row number 



In [0]:
# Imports 
import pandas as pd
from datetime import date

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
base_dir = "/content/drive/My Drive/CbcColab_work/"

In [0]:
# Load in the cleaned data
cbc_cir_df = pd.read_csv("/content/drive/My Drive/CbcColab_work/cleaned_cbc_usa.csv", encoding ="latin_1")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
print(cbc_cir_df.shape)
cbc_cir_df.head(53)

(89568, 48)


,Unnamed__0,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,field_hours,feeder_hours,nocturnal_hours,field_distance,nocturnal_distance,distance_units,min_temp,max_temp,temp_unit,min_wind,max_wind,wind_unit,min_snow,max_snow,snow_unit,am_cloud,pm_cloud,am_rain,pm_rain,am_snow,pm_snow,field_distance_imperial,field_distance_metric,nocturnal_distance_imperial,nocturnal_distance_metric,min_snow_imperial,min_snow_metric,max_snow_metric,max_snow_imperial,min_temp_imperial,max_temp_imperial,min_temp_metric,max_temp_metric,min_wind_metric,max_wind_metric,min_wind_imperial,max_wind_imperial
0,2,Pacific Grove,US-CA,36.616700,-121.916700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,Pueblo,US-CO,38.175251,-104.519575,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,Bristol,US-CT,41.671800,-72.949500,1901,1900-12-25,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,Norwalk,US-CT,41.116700,-73.400000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,Glen Ellyn,US-IL,41.883300,-88.066700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7,Baldwin,US-LA,29.833300,-91.550000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,8,Belmont,US-MA,42.383300,-71.166700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,9,Greater Boston,US-MA,42.397309,-71.095501,1901,1900-12-26,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10,Winchester,US-MA,42.450000,-71.133300,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,11,La Grange,US-MO,40.050000,-91.500000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# Drop all the locations that are not in the united states 
indexNamesNUSA = cbc_cir_df[~cbc_cir_df['country_state'].str.contains("US-")].index
 
# Delete these row indexes from dataFrame
cbc_cir_df.drop(indexNamesNUSA , inplace=True)

In [0]:
print(cbc_cir_df.shape)
cbc_cir_df.head(10)

(89568, 48)


,Unnamed__0,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,field_hours,feeder_hours,nocturnal_hours,field_distance,nocturnal_distance,distance_units,min_temp,max_temp,temp_unit,min_wind,max_wind,wind_unit,min_snow,max_snow,snow_unit,am_cloud,pm_cloud,am_rain,pm_rain,am_snow,pm_snow,field_distance_imperial,field_distance_metric,nocturnal_distance_imperial,nocturnal_distance_metric,min_snow_imperial,min_snow_metric,max_snow_metric,max_snow_imperial,min_temp_imperial,max_temp_imperial,min_temp_metric,max_temp_metric,min_wind_metric,max_wind_metric,min_wind_imperial,max_wind_imperial
0,2,Pacific Grove,US-CA,36.616700,-121.916700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,Pueblo,US-CO,38.175251,-104.519575,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,Bristol,US-CT,41.671800,-72.949500,1901,1900-12-25,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,Norwalk,US-CT,41.116700,-73.400000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,Glen Ellyn,US-IL,41.883300,-88.066700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7,Baldwin,US-LA,29.833300,-91.550000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,8,Belmont,US-MA,42.383300,-71.166700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,9,Greater Boston,US-MA,42.397309,-71.095501,1901,1900-12-26,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10,Winchester,US-MA,42.450000,-71.133300,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,11,La Grange,US-MO,40.050000,-91.500000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Prep the Closest Station File 
The closest stations (as determanted by Lat Long) was pulled via python script by slack user mhinq.
It is called output.csv in the github. 

Below we will:
1. Load the File
<!-- 2. Remove the duplicate enteries (Mostly so I can export a shorter file later)  --> NOT Preforming anymore becuase even duplicate locations can have different statuons over the years
3. Add in percision Decimals to the Lat Long Cordinates 

In [0]:
# Load the Closest Station File 
clos_st_nm = pd.read_csv('/content/drive/My Drive/CbcColab_work/closest_stations_geoyear_20200127.csv', header = 0, encoding ="latin_1")

print(clos_st_nm.shape)
clos_st_nm.head()

(89568, 9)


,cbc_count_year,circle_lat,circle_lng,circle_name,circle_num,closest_station_id,closest_station_lat,closest_station_lng,distance
0,1966,41.107008,-72.334595,L.I.: Orient,19423,USC00303464,41.1019,-72.3731,3276.750141
1,1961,41.384157,-82.189719,Elyria-Lorain,15748,USC00332599,41.3814,-82.0558,11180.310163
2,1946,42.109511,-79.228425,Jamestown,8279,USC00304206,42.1000,-79.2500,2070.964211
3,1993,37.967164,-75.366874,Chincoteague N.W.R.,55608,USW00093739,37.9406,-75.4631,8941.538205
4,2000,29.634400,-96.366700,Attwater Prairie Chicken N.W.R.,68141,USC00411911,29.6989,-96.5731,21198.724723


In [0]:
clos_st_nm['circle_coordinates'] = "(" + clos_st_nm.circle_lat.map(str) + ", " + clos_st_nm.circle_lng.map(str)

In [0]:
# Limit the Stations down to just the unique station cordinates 
#clos_st_nm.drop_duplicates(subset='circle_coordinates', keep="first", inplace = True)
#print(clos_st_nm.shape)

#clos_st_nm.head(20)



In [0]:
# The Station cordinates in the closest station file (output.csv) are shorter/less percise 
# than the data that is in the main cbc file. We will restore the precision of these decimals 

# Split the circle coordinates into two columns for a sec 
temp_latlon = clos_st_nm["circle_coordinates"].str.split(",", n = 1, expand = True) 
temp_latlon.head()

# Remove the "(" and ")"
temp_latlon[0].replace(regex=True,inplace=True,to_replace=r'\(',value=r'')
temp_latlon[1].replace(regex=True,inplace=True,to_replace=r'\)',value=r'')
temp_latlon.head(20)

##Force these digits to have 6 decimals 
temp_latlon = temp_latlon.astype(float).round(6).copy(False)
to_8digit_str = lambda flt: str(flt).ljust(8,"0")
to_9digit_str = lambda flt: str(flt).ljust(9,"0")

temp_latlon[0] = temp_latlon[0].map(to_8digit_str)
temp_latlon[1] = temp_latlon[1].map(to_9digit_str)


## recombine into tuple 
temp_latlon["circle_coordinates"] = "(" + temp_latlon[0].astype(str) + ", " + temp_latlon[1].astype(str) + ")" 

temp_latlon.head(20)

## Save the result back to the closest station data 
clos_st_nm["circle_coordinates"] = temp_latlon["circle_coordinates"]
clos_st_nm.head(20)


,cbc_count_year,circle_lat,circle_lng,circle_name,circle_num,closest_station_id,closest_station_lat,closest_station_lng,distance,circle_coordinates
0,1966,41.107008,-72.334595,L.I.: Orient,19423,USC00303464,41.1019,-72.3731,3276.750141,"(41.107008, -72.334595)"
1,1961,41.384157,-82.189719,Elyria-Lorain,15748,USC00332599,41.3814,-82.0558,11180.310163,"(41.384157, -82.189719)"
2,1946,42.109511,-79.228425,Jamestown,8279,USC00304206,42.1000,-79.2500,2070.964211,"(42.109511, -79.228425)"
3,1993,37.967164,-75.366874,Chincoteague N.W.R.,55608,USW00093739,37.9406,-75.4631,8941.538205,"(37.967164, -75.366874)"
4,2000,29.634400,-96.366700,Attwater Prairie Chicken N.W.R.,68141,USC00411911,29.6989,-96.5731,21198.724723,"(29.63440, -96.36670)"
5,1984,37.921391,-122.564311,Marin County (southern),39943,USC00046027,37.8978,-122.5689,2654.663788,"(37.921391, -122.564311)"
6,1940,37.733300,-89.216700,Carbondale,6498,USC00111265,37.7308,-89.1658,4486.138030,"(37.73330, -89.21670)"
7,1981,34.766709,-84.966702,Dalton,35647,USC00092493,34.7700,-84.8872,7273.192911,"(34.766709, -84.966702)"
8,1979,32.301632,-110.973489,Tucson Valley,32682,USC00028796,32.2817,-110.9442,3535.353606,"(32.301632, -110.973489)"
9,2013,41.751812,-87.973816,Lisle Arboretum,95455,US1ILDP0118,41.7431,-87.9746,971.185532,"(41.751812, -87.973816)"


# Prep Merge Between CBC Circle Data and the Closest Station Data 

In [0]:
# Create a Key in cbc_cir_df as the Lat Lon as a tuble
cbc_cir_df['station_key'] = "(" + cbc_cir_df['lat'].round(6).map(to_8digit_str) + ", " + cbc_cir_df['lon'].round(6).map(to_9digit_str) + ")" 


In [0]:
print(cbc_cir_df.shape)
cbc_cir_df.head(53)

(89568, 49)


,Unnamed__0,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,field_hours,feeder_hours,nocturnal_hours,field_distance,nocturnal_distance,distance_units,min_temp,max_temp,temp_unit,min_wind,max_wind,wind_unit,min_snow,max_snow,snow_unit,am_cloud,pm_cloud,am_rain,pm_rain,am_snow,pm_snow,field_distance_imperial,field_distance_metric,nocturnal_distance_imperial,nocturnal_distance_metric,min_snow_imperial,min_snow_metric,max_snow_metric,max_snow_imperial,min_temp_imperial,max_temp_imperial,min_temp_metric,max_temp_metric,min_wind_metric,max_wind_metric,min_wind_imperial,max_wind_imperial,station_key
0,2,Pacific Grove,US-CA,36.616700,-121.916700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(36.61670, -121.9167)"
1,3,Pueblo,US-CO,38.175251,-104.519575,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(38.175251, -104.519575)"
2,4,Bristol,US-CT,41.671800,-72.949500,1901,1900-12-25,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(41.67180, -72.94950)"
3,5,Norwalk,US-CT,41.116700,-73.400000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(41.11670, -73.40000)"
4,6,Glen Ellyn,US-IL,41.883300,-88.066700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(41.88330, -88.06670)"
5,7,Baldwin,US-LA,29.833300,-91.550000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(29.83330, -91.55000)"
6,8,Belmont,US-MA,42.383300,-71.166700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(42.38330, -71.16670)"
7,9,Greater Boston,US-MA,42.397309,-71.095501,1901,1900-12-26,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(42.397309, -71.095501)"
8,10,Winchester,US-MA,42.450000,-71.133300,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(42.45000, -71.13330)"
9,11,La Grange,US-MO,40.050000,-91.500000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(40.05000, -91.50000)"


# Merge the dataframes based on NOT lat long 
Row ID number instead


In [0]:
# Merge the dataframes based on lat long 
results = pd.merge(cbc_cir_df, clos_st_nm[['circle_coordinates','closest_station_id', 'closest_station_lat', 'closest_station_lng', 'distance', 'circle_num']], how = "left", left_on='Unnamed__0', right_on='circle_num')


In [0]:
print(results.shape)
results.head(50)

(89568, 55)


,Unnamed__0,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,field_hours,feeder_hours,nocturnal_hours,field_distance,nocturnal_distance,distance_units,min_temp,max_temp,temp_unit,min_wind,max_wind,wind_unit,min_snow,max_snow,snow_unit,am_cloud,pm_cloud,am_rain,pm_rain,am_snow,pm_snow,field_distance_imperial,field_distance_metric,nocturnal_distance_imperial,nocturnal_distance_metric,min_snow_imperial,min_snow_metric,max_snow_metric,max_snow_imperial,min_temp_imperial,max_temp_imperial,min_temp_metric,max_temp_metric,min_wind_metric,max_wind_metric,min_wind_imperial,max_wind_imperial,station_key,circle_coordinates,closest_station_id,closest_station_lat,closest_station_lng,distance,circle_num
0,2,Pacific Grove,US-CA,36.616700,-121.916700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(36.61670, -121.9167)","(36.61670, -121.9167)",USC00047916,36.9878,-121.9994,41928.062371,2
1,3,Pueblo,US-CO,38.175251,-104.519575,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(38.175251, -104.519575)","(38.175251, -104.519575)",USW00023068,38.2725,-104.6089,13338.679404,3
2,4,Bristol,US-CT,41.671800,-72.949500,1901,1900-12-25,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(41.67180, -72.94950)","(41.67180, -72.94950)",USC00061536,41.8000,-72.9333,14322.471705,4
3,5,Norwalk,US-CT,41.116700,-73.400000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(41.11670, -73.40000)","(41.11670, -73.40000)",USC00065892,41.1333,-73.4500,4578.046523,5
4,6,Glen Ellyn,US-IL,41.883300,-88.066700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(41.88330, -88.06670)","(41.88330, -88.06670)",USC00119221,41.8128,-88.0728,7857.728111,6
5,7,Baldwin,US-LA,29.833300,-91.550000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(29.83330, -91.55000)","(29.83330, -91.55000)",USC00163313,29.8233,-91.5442,1245.124892,7
6,8,Belmont,US-MA,42.383300,-71.166700,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(42.38330, -71.16670)","(42.38330, -71.16670)",USC00191097,42.3833,-71.1167,4107.877116,8
7,9,Greater Boston,US-MA,42.397309,-71.095501,1901,1900-12-26,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(42.397309, -71.095501)","(42.397309, -71.095501)",USC00191097,42.3833,-71.1167,2336.790124,9
8,10,Winchester,US-MA,42.450000,-71.133300,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(42.45000, -71.13330)","(42.45000, -71.13330)",USC00191097,42.3833,-71.1167,7542.981159,10
9,11,La Grange,US-MO,40.050000,-91.500000,1901,1900-12-25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Miles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(40.05000, -91.50000)","(40.05000, -91.50000)",USC00231275,40.1433,-91.5158,10464.129047,11


In [0]:
# There are 0 sites that I cannot Match, might just do it manually cuz I'm Lazy, but not right now 
results['closest_station_id'].isna().sum(axis = 0)

0

In [0]:
# These are the stations that were not matched to a Closest Station 
results[~results.closest_station_id.notnull()]

,Unnamed__0,circle_name,country_state,lat,lon,count_year,count_date,n_field_counters,n_feeder_counters,min_field_parties,max_field_parties,field_hours,feeder_hours,nocturnal_hours,field_distance,nocturnal_distance,distance_units,min_temp,max_temp,temp_unit,min_wind,max_wind,wind_unit,min_snow,max_snow,snow_unit,am_cloud,pm_cloud,am_rain,pm_rain,am_snow,pm_snow,field_distance_imperial,field_distance_metric,nocturnal_distance_imperial,nocturnal_distance_metric,min_snow_imperial,min_snow_metric,max_snow_metric,max_snow_imperial,min_temp_imperial,max_temp_imperial,min_temp_metric,max_temp_metric,min_wind_metric,max_wind_metric,min_wind_imperial,max_wind_imperial,station_key,circle_coordinates,closest_station_id,closest_station_lat,closest_station_lng,distance,circle_num


In [0]:
# Save the File 

results.to_csv(base_dir + "cbc_cleaned_usa_statid.csv", index = False)